In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import datetime as dt
from technicals.indicators import MACD
from simulation.guru_tester import GuruTester

In [3]:
# the custom part of our strategy
BUY = 1
SELL = -1
NONE = 0

def apply_signal(row):
    if row.direction == BUY and row.mid_l > row.EMA_100:
        return BUY
    if row.direction == SELL and row.mid_h < row.EMA_100:
        return SELL
    return NONE        

In [4]:
# loading the dfs
pair = "EUR_USD"
#load up data to their dfs
df_an = pd.read_pickle(f"../data/{pair}_H1.pkl")
df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
# reset indexes on the data
df_an.reset_index(drop=True, inplace=True)
df_m5.reset_index(drop=True, inplace=True)

In [5]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2016-01-07 00:00:00+00:00,542,1.07764,1.07832,1.07744,1.07778,1.07757,1.07823,1.07735,1.07770,1.07772,1.07840,1.07752,1.07787
1,2016-01-07 01:00:00+00:00,3167,1.07776,1.08100,1.07748,1.08029,1.07768,1.08092,1.07740,1.08020,1.07784,1.08109,1.07756,1.08038
2,2016-01-07 02:00:00+00:00,1567,1.08026,1.08176,1.07996,1.08152,1.08018,1.08169,1.07987,1.08144,1.08035,1.08184,1.08005,1.08159
3,2016-01-07 03:00:00+00:00,914,1.08156,1.08257,1.08150,1.08187,1.08147,1.08249,1.08142,1.08178,1.08164,1.08265,1.08157,1.08196
4,2016-01-07 04:00:00+00:00,649,1.08190,1.08256,1.08156,1.08236,1.08182,1.08247,1.08147,1.08228,1.08199,1.08264,1.08163,1.08245


In [6]:
# apply MACD and EMA100
df_an = MACD(df_an)
df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
# drop any NaN/Null/na values
df_an.dropna(inplace=True)
# reset the index
df_an.reset_index(drop=True, inplace=True)


In [7]:
# check df_an to ensure theres MACD and EMA100 columns
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100
0,2016-01-13 03:00:00+00:00,450,1.08290,1.08324,1.08247,1.08250,1.08282,1.08316,1.08240,1.08243,1.08297,1.08331,1.08254,1.08258,-0.001062,-0.000931,-0.000131,1.086813
1,2016-01-13 04:00:00+00:00,239,1.08247,1.08314,1.08243,1.08280,1.08239,1.08307,1.08235,1.08272,1.08255,1.08321,1.08251,1.08287,-0.001075,-0.000960,-0.000115,1.086722
2,2016-01-13 05:00:00+00:00,518,1.08279,1.08352,1.08272,1.08294,1.08272,1.08344,1.08266,1.08287,1.08286,1.08360,1.08279,1.08301,-0.001062,-0.000981,-0.000082,1.086635
3,2016-01-13 06:00:00+00:00,355,1.08290,1.08379,1.08290,1.08332,1.08283,1.08373,1.08282,1.08324,1.08298,1.08385,1.08297,1.08341,-0.001009,-0.000986,-0.000023,1.086560
4,2016-01-13 07:00:00+00:00,1647,1.08332,1.08366,1.08134,1.08162,1.08324,1.08358,1.08126,1.08155,1.08340,1.08375,1.08142,1.08169,-0.001092,-0.001007,-0.000085,1.086448


In [8]:
# calculate the MA cross

In [9]:
# columns in df_an
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'MACD', 'SIGNAL',
       'HIST', 'EMA_100'],
      dtype='object')

In [10]:
# apply BUY/SELL cross
def apply_cross(row):
    if row.macd_delta > 0 and row.macd_delta_prev < 0:
        return BUY
    if row.macd_delta < 0 and row.macd_delta_prev > 0:
        return SELL
    return NONE

In [11]:
# calc the 'delta' which is the diff between MACD line and SIGNAL line
df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
# create a delta_prev column which shows the prev row value
df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
# get the trade direction: BUY/SELL or NONE
df_an['direction'] = df_an.apply(apply_cross, axis=1)

In [12]:
# check we got all 3 types of directions:
df_an.direction.unique()

array([ 0,  1, -1], dtype=int64)

In [13]:
# check number of BUYs (1), SELLs(-1) and NONEs(0)
df_an.direction.value_counts()

 0    34175
 1     1473
-1     1472
Name: direction, dtype: int64

In [14]:
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,ask_h,ask_l,ask_c,MACD,SIGNAL,HIST,EMA_100,macd_delta,macd_delta_prev,direction
0,2016-01-13 03:00:00+00:00,450,1.08290,1.08324,1.08247,1.08250,1.08282,1.08316,1.08240,1.08243,...,1.08331,1.08254,1.08258,-0.001062,-0.000931,-0.000131,1.086813,-0.000131,NaN,0
1,2016-01-13 04:00:00+00:00,239,1.08247,1.08314,1.08243,1.08280,1.08239,1.08307,1.08235,1.08272,...,1.08321,1.08251,1.08287,-0.001075,-0.000960,-0.000115,1.086722,-0.000115,-0.000131,0
2,2016-01-13 05:00:00+00:00,518,1.08279,1.08352,1.08272,1.08294,1.08272,1.08344,1.08266,1.08287,...,1.08360,1.08279,1.08301,-0.001062,-0.000981,-0.000082,1.086635,-0.000082,-0.000115,0
3,2016-01-13 06:00:00+00:00,355,1.08290,1.08379,1.08290,1.08332,1.08283,1.08373,1.08282,1.08324,...,1.08385,1.08297,1.08341,-0.001009,-0.000986,-0.000023,1.086560,-0.000023,-0.000082,0
4,2016-01-13 07:00:00+00:00,1647,1.08332,1.08366,1.08134,1.08162,1.08324,1.08358,1.08126,1.08155,...,1.08375,1.08142,1.08169,-0.001092,-0.001007,-0.000085,1.086448,-0.000085,-0.000023,0


In [15]:
# this will load up whichever pair you want, example, EUR_USD
def run_pair(pair, time_d=1):
    #load up data to their dfs
    df_an = pd.read_pickle(f"../data/{pair}_H{time_d}.pkl")
    df_m5 = pd.read_pickle(f"../data/{pair}_M5.pkl")
    # reset indexes on the data
    df_an.reset_index(drop=True, inplace=True)
    df_m5.reset_index(drop=True, inplace=True)
    # the custom part. the strategies.
    df_an = MACD(df_an)
    df_an['EMA_100'] = df_an.mid_c.ewm(span=100, min_periods=100).mean()
    
    # add the 'direction' column
    df_an['macd_delta'] = df_an.MACD - df_an.SIGNAL
    df_an['macd_delta_prev'] = df_an.macd_delta.shift(1)
    df_an['direction'] = df_an.apply(apply_cross, axis=1)
    
    # make a slimmer df with columns that we need AS A COPY
    our_cols = ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c',
            'bid_o', 'bid_h', 'bid_l', 'bid_c', 
            'ask_o', 'ask_h', 'ask_l', 'ask_c',
                'direction', 'EMA_100']
    df_slim = df_an[our_cols].copy()
    # drop any NaN/Null/na values
    df_slim.dropna(inplace=True)
    
    # reset the index
    df_slim.reset_index(drop=True, inplace=True)
    # finally we create an instance of GuruTester class
    gt = GuruTester(
        df_slim,
        apply_signal,
        df_m5,
        use_spread=True,
        time_d=time_d
    )
    
    # after the intance is created, we run the test
    gt.run_test()
    # after running test, we return the result
    return gt.df_results
    

In [16]:
# we will run 4 pairs and seetheir results in the output
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=1)))

In [17]:
# show those results next their respective pair
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -367.0
EUR_USD 22.0
GBP_JPY 92.0
USD_CAD -27.0


In [18]:
# do a test with H4
res = []
for p in ["AUD_NZD", "EUR_USD", "GBP_JPY", "USD_CAD" ]:
    res.append(dict(pair=p,res=run_pair(p, time_d=4)))
for r in res:
    print(r['pair'], r['res'].result.sum())

AUD_NZD -28.5
EUR_USD 19.5
GBP_JPY 36.5
USD_CAD -24.0
